In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers


In [2]:
celsius_q    = np.array([-40, -10,  0,  8, 15, 22,  38],  dtype=float)
fahrenheit_a = np.array([-40,  14, 32, 46, 59, 72, 100],  dtype=float)

for i,c in enumerate(celsius_q):
  print("{} degrees Celsius = {} degrees Fahrenheit".format(c, fahrenheit_a[i]))

-40.0 degrees Celsius = -40.0 degrees Fahrenheit
-10.0 degrees Celsius = 14.0 degrees Fahrenheit
0.0 degrees Celsius = 32.0 degrees Fahrenheit
8.0 degrees Celsius = 46.0 degrees Fahrenheit
15.0 degrees Celsius = 59.0 degrees Fahrenheit
22.0 degrees Celsius = 72.0 degrees Fahrenheit
38.0 degrees Celsius = 100.0 degrees Fahrenheit


In [6]:
model = tf.keras.Sequential([
  layers.Dense(units=1, input_shape=[1])
])
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])

In [7]:
model.fit(celsius_q, fahrenheit_a, epochs=500, verbose=False)

In [8]:
model.predict([100.0])

1/1 [==============================] - 0s 55ms/step


array([[-55.795628]], dtype=float32)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

# CBAM (Convolutional Block Attention Module) Layer
class CBAM(layers.Layer):
    def __init__(self):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention()
        self.spatial_attention = SpatialAttention()
    
    def call(self, inputs):
        channel_att = self.channel_attention(inputs)
        spatial_att = self.spatial_attention(inputs)
        return tf.multiply(inputs, tf.expand_dims(tf.expand_dims(channel_att, axis=1), axis=1)) * spatial_att

# Channel Attention Layer
class ChannelAttention(layers.Layer):
    def __init__(self):
        super(ChannelAttention, self).__init__()
        self.global_avgpool = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        avg_pool = self.global_avgpool(inputs)
        dense1_out = self.dense1(avg_pool)
        attention = self.dense2(dense1_out)
        return attention

# Linear Attention Layer
class LinearAttention(layers.Layer):
    def __init__(self):
        super(LinearAttention, self).__init__()
        self.dense = layers.Dense(1)
    
    def call(self, inputs):
        return tf.nn.sigmoid(self.dense(inputs))

# NonLinear Attention Layer
class NonLinearAttention(layers.Layer):
    def __init__(self):
        super(NonLinearAttention, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        dense1_out = self.dense1(inputs)
        attention = self.dense2(dense1_out)
        return attention

# Configure model function
def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy']):
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

# Train model function
def train_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32, model_name='model', log_dir='./logs'):
    tensorboard_callback = TensorBoard(log_dir=log_dir)
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time
    predictions = model.predict(X_test)
    return model, history, predictions, training_time

# Test model function
def test_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    return predictions

# Calculate metrics function
def calculate_metrics(y_true, y_pred):
    threshold = 0.5
    correct_predictions = np.sum(np.abs(y_true - y_pred.flatten()) <= threshold)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    f1 = f1_score(y_true, y_pred.round(), average='micro')
    return accuracy, mse, mae, rmse, f1

# Example usage
mlp_config = [
    ('dense', 64, 'linear', dset_features.shape[1]),  # Hidden layer 1 with input shape
    ('dense', 32, 'selu'),  # Hidden layer 2
    ('dense', 1, 'linear')  # Output layer
]   

model = configure_model(mlp_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    # Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test, model_name='multi_layer_perceptron')
    # Test the model

predictions = test_model(model, X_test, y_test)
    # Calculate metrics
accuracy, mse, mae, rmse, f1 = calculate_metrics(y_test, predictions)
print("Metrics:")
print(f"Accuracy: {accuracy}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"F1 Score: {f1}")
